In [0]:
# spark.conf.set("fs.azure.account.key.adlsairqualitypoc.dfs.core.windows.net", "wyOLWhyzy7LbDaqB/GYL602VZsO98fuB5Elr6qUQBHV2uswoPxlHEfTyYS1bTRvKYQD2s/lrk2Uk+AStSn6bNg==")

In [0]:
# %sql
# SELECT
#   state,
#   date,
#   avg_aqi
# FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_aqi`
# WHERE pollutant_id = 'PM2.5'
# ORDER BY date;

In [0]:
%sql
--AQI Trend by State
SELECT
    date,
    state,
    pollutant_id,
    avg_aqi
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_state`
ORDER BY state, date;

In [0]:
%sql
--AQI Trend by City
SELECT
    date,
    state,
    city,
    pollutant_id,
    avg_aqi
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_city`
ORDER BY date;

In [0]:
%sql
--Pollutant Comparison
SELECT
    date,
    pollutant_id,
    avg_aqi
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_state`
WHERE state = 'Delhi'
ORDER BY date;

In [0]:
%sql
--Hourly State Trend
SELECT
    hour_ts,
    state,
    pollutant_id,
    avg_aqi
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_hourly_state`
ORDER BY hour_ts;

In [0]:
%sql
--Threshold Breach Alerts
SELECT
    state,
    city,
    pollutant_id,
    event_ts,
    pollutant_avg
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/fact_threshold_breach`
ORDER BY event_ts DESC;

In [0]:
%sql
--Historical vs Recent Comparison
WITH classified AS (
    SELECT
        CASE 
            WHEN date BETWEEN date_sub(current_date(), 7) AND current_date()
            THEN 'Recent (Last 7 Days)'
            ELSE 'Historical'
        END AS period,
        state,
        pollutant_id,
        avg_aqi
    FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_state`
)

SELECT
    period,
    state,
    pollutant_id,
    AVG(avg_aqi) AS avg_aqi
FROM classified
GROUP BY period, state, pollutant_id
ORDER BY period;

In [0]:
%sql
--Top 10 Most Polluted Cities (Latest Day)
SELECT
    state,
    city,
    avg_aqi
FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_city`
WHERE date = (
    SELECT MAX(date)
    FROM delta.`abfss://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/agg_daily_city`
)
ORDER BY avg_aqi DESC
LIMIT 10;